<a href="https://colab.research.google.com/github/ShinnosukeAsaga/Mathematical-Modeling/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
#Parameter setup
road_length =100   #Length of the road
spead_max =5        #Maximum　Speed
car_density  = 0.3   #Random Slowdown Chance
steps =200          #Simulation Steps
p_slow = 0.2        #The probability of randomly decelerating

In [ ]:
# Initialize (-1 is an empty cell)
num_cars = int(road_length * p_slow  )
road = -np.ones(road_length, dtype=int)
car_positions = np.random.choice(road_length, num_cars, replace=False)
road[car_positions] = np.random.randint(0, spead_max + 1, size=num_cars)

In [ ]:
# Space-time data
space_time = np.zeros((steps, road_length), dtype=int)
road_history = []


### Nagel-Schreckenberg Model: Update Rules

The update for each vehicle *i* proceeds in four steps at each time *t*:

---

#### 1. Acceleration

$$
v_{i}(t+1) = \min \left\{ v_{i}(t) + 1,\ v_{\text{max}} \right\}
$$

- $ v_{i}(t)$　 Speed of vehicle *i* at time
- $ v_{\text{max}} $: Maximum allowed speed

---

#### 2. Slowing Down (Due to Other Vehicles)

$$
v_{i}(t+1) = \min \left\{ v_{i}(t+1),\ d_{i}(t) \right\}
$$

- $ d_{i}(t) $: Number of empty cells in front of vehicle *i*  
  (i.e., distance to the next vehicle in front minus 1)

This prevents collisions by ensuring the vehicle does not move farther than the gap.

---

#### 3. Randomization (Stochastic Braking)

With probability $ p $, the speed is reduced by 1:

$$
v_{i}(t+1) =
\begin{cases}
\max \left\{ v_{i}(t+1) - 1,\ 0 \right\}, & \text{with probability } p \\
v_{i}(t+1), & \text{with probability } 1 - p
\end{cases}
$$

- $ p $: Random slowdown probability (e.g., 0.25)

This introduces variability in driver behavior.

---

#### 4. Vehicle Movement

After updating speed, the vehicle moves:

$$
x_{i}(t+1) = x_{i}(t) + v_{i}(t+1)
$$

- $ x_{i}(t) $: Position of vehicle *i* at time \( t \)

The vehicle advances by its updated speed.

---


In [ ]:
# Simulation
for t in range(steps):
    space_time[t] = road >= 0
    road_history.append(road >= 0)

    new_road = -np.ones(road_length, dtype=int)

    for i in range(road_length):
        if road[i] >= 0:
            v = road[i]
            #Rule1 acceleration
            v = min(v + 1, spead_max)
            #Rule2 Slow down
            for d in range(1, v + 1):
                if road[(i + d) % road_length] >= 0:
                    v = d - 1
                    break
            #Rule3 Random Deceleration
            if np.random.rand() < p_slow:
                v = max(v - 1, 0)
            #Rule4 Movement
            new_pos = (i + v) % road_length
            new_road[new_pos] = v

    road = new_road

In [ ]:
# Save as PNG
plt.figure(figsize=(10, 6))
plt.imshow(space_time, cmap='gray_r', interpolation='none', aspect='auto')
plt.title("Nagel-Schreckenberg Model Spacetime Diagram", fontname='IPAPGothic')
plt.xlabel("Position", fontname='IPAPGothic')
plt.ylabel("time", fontname='IPAPGothic')
plt.tight_layout()
plt.savefig("nagel_schreckenberg_space_time.png", dpi=300)
plt.close()

In [ ]:
# MP4 Animation Generation
fig, ax = plt.subplots()
img = ax.imshow(road_history[0].reshape(1, -1), cmap='gray_r', interpolation='none', aspect='auto')
ax.set_title("Nagel-Schreckenberg Animation", fontname='IPAPGothic')
ax.set_yticks([])
ax.set_xlabel("Position", fontname='IPAPGothic')

def update(frame):
    img.set_array(road_history[frame].reshape(1, -1))
    return [img]

ani = animation.FuncAnimation(fig, update, frames=steps, interval=100, blit=True)

In [ ]:
# Save as movie (FFmpeg is required.)
ani.save("nagel_schreckenberg_animation.gif", writer='pillow', dpi=200)


plt.close()
